# 一、 简介
- 参考：https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt
## 1. 什么是Causal Language Model(CLM)？
这个概念可以和Mask Language model(MLM)进行对比：
- MLM：是将某些token随机替换为[MASK]这一个token；训练时，除了被MASK的token外，其他所有token都能看到，没有先后顺序；代表模型是Bert
```
The goal of life is [Mask].  （注意[MASK]后还有一个“.”token）

[MASK]: [life: 0.109, survival: 0.039, ....] 
```
- CLM：是一种自回归模型；有先后顺序，对于当前token，相当于当前以及之后的所有token都被mask了；代表模型是GPT2
```
My name is Sylvain and I like to

My name is Sylvain and I like to talk about music and comics and film. ....
```

MLM和CLM共同点都是自监督模型（无监督），而Translation Language Modeling (TLM，翻译语言模型)是监督模型，需要标注数据。CLM也是一种Decoder-only的LLM，即Transformer架构中，相当于只有Decoder，没有Encoder（区别于Bert）。MLM、CLM、TLM都属于生成式模型。

按照目前的趋势，目前的LLM的含义更接近于CLM，因此不管是pre-train LLM和post-train LLM，通常都指代的是CLM。

由于有强烈的先后关系的因果关系，因此能够很好的用来做代码生成、音符生成、文章生成等任务。本章微调一个CLM来做code generation



In [2]:
import os
# 获取当前工作目录
current_directory = os.getcwd()
print("当前工作目录:", current_directory)

# 设置新的工作目录
new_directory = "/mnt/d/code/llm/llm-course"
os.chdir(new_directory)

# 再次获取当前工作目录，确认是否更改成功
current_directory = os.getcwd()
print("新的工作目录:", current_directory)

当前工作目录: /mnt/d/code/llm/llm-course/mynotes
新的工作目录: /mnt/d/code/llm/llm-course


# 二、 微调CLM得到一个code generation model

本节只考虑python代码，并且只考虑python代码的子集，包含了matplotlib, seaborn, pandas, scikit-learn这几个库

## 1. 获取数据
数据集使用的是[codeparrot](https://huggingface.co/datasets/transformersbook/codeparrot),
由于只训练python代码子集，因此需要对codeparrot数据集过滤。

### (1) python代码子集数据过滤

In [3]:
# 对codeparrot数据集过滤，只包含matplotlib, seaborn, pandas, scikit-learn这几个库的数据
def any_keyword_in_string(string, keywords):
    for keyword in keywords:
        if keyword in string:
            return True
    return False

In [4]:
# 过滤样例
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]
example_1 = "import numpy as np"    # 过滤掉
example_2 = "import pandas as pd"   # 保留

print(
    any_keyword_in_string(example_1, filters), any_keyword_in_string(example_2, filters)
)

False True


In [5]:
from collections import defaultdict
from tqdm import tqdm
from datasets import Dataset

# 流式过滤数据集
def filter_streaming_dataset(dataset, filters):
    filtered_dict = defaultdict(list)
    total = 0
    for sample in tqdm(iter(dataset)):
        total += 1
        if any_keyword_in_string(sample["content"], filters):
            for k, v in sample.items():
                filtered_dict[k].append(v)
    print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering.")
    return Dataset.from_dict(filtered_dict)

进行数据过滤

In [ ]:
# 数据过滤
# This cell will take a very long time to execute, so you should skip it and go to
# the next one!
from datasets import load_dataset

split = "train"  # "valid"
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]

data = load_dataset(f"transformersbook/codeparrot-{split}", split=split, streaming=True)

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

18593791it [8:28:09, 609.84it/s] 


3.26% of data after filtering.


In [1]:
# 过滤数据
filtered_data = filter_streaming_dataset(data, filters)

NameError: name 'filter_streaming_dataset' is not defined

### (2) 过滤后的数据集加载

In [ ]:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(50000)),  # 数据集shuffle并减小样本数
        "valid": ds_valid.shuffle().select(range(500))
    }
)

raw_datasets